# PINNs para el sistema masa–resorte–amortiguador

**Autor:** David Ortiz — 2025

Accede al trabajo fundacional de las PINNs [aquí](https://www.sciencedirect.com/science/article/pii/S0021999118307125).

Este tutorial está inspirado en [este](https://github.com/benmoseley/harmonic-oscillator-pinn-workshop) workshop, elaborado por Ben Moseley

### Introducción
Las Redes Neuronales Informadas por la Física (PINNs) incorporan ecuaciones gobernantes en la función de pérdida mediante diferenciación automática, reduciendo la dependencia de grandes volúmenes de datos y mejorando la interpretabilidad. En esta actividad aplicaremos PINNs al modelo lineal clásico **masa–resorte–amortiguador**, para el cual existe solución analítica, lo que permitirá verificar cuantitativamente el desempeño del enfoque.

### Objetivos de de aprendizaje
- Implementar una PINN desde cero en PyTorch para resolver el sistema masa-resorte-amortiguador siguiendo las 6 etapas vistas
- Comprender los diferentes componentes de la función de pérdida de una PINN (residuo de la EDO, condiciones iniciales y de frontera
- Entender en mayor detalle cómo se entrenan las PINNs y cómo usar diferenciación automática para calcular derivadas de redes neuronales

### Resumen de la Actividad
Construir una PINN para el sistema masa-resorte-amortiguador, basandonos en los 6 pasos vistos en el taller teórico:

<img src="../figures/pinns_new_scheme.png" width="800" height="400">

- (1) formular el modelo matemático y su solución analítica; 
- (2) definir el dominio (temporal); 
- (3) implementar una ANN como surrogate de $x(t)$;
- (4) Diferenciación automática
- (5) diseñar la loss con términos físicos (residuo de la EDO e inicio/condición inicial) usando autograd; 
- (6) fijar un optimizador (Adam);

Finalmete, ejecutar el ciclo de entrenamiento y prueba, comparando contra la solución exacta.

## 0. Importamos funciones 

In [ ]:
%matplotlib inline

In [ ]:
# Import NumPy for numerical operations
import numpy as np
# Import PyTorch for building and training neural networks
import torch
import torch.nn as nn
import torch.optim as optim
# Import Matplotlib for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
# Import the time module to time our training process
import time
# Ignore Warning Messages
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Setup (device + plots)
def get_device() -> str:
    return "cuda" if torch.cuda.is_available() else \
           "mps"  if torch.backends.mps.is_available() else "cpu"

def set_mpl_style(gray: str = "#5c5c5c") -> None:
    mpl.rcParams.update({
        "image.cmap": "viridis",
        "text.color": gray, "xtick.color": gray, "ytick.color": gray,
        "axes.labelcolor": gray, "axes.edgecolor": gray,
        "axes.spines.right": False, "axes.spines.top": False,
        "axes.formatter.use_mathtext": True, "axes.unicode_minus": False,
        "font.size": 15, "interactive": False, "font.family": "sans-serif",
        "legend.loc": "best", "text.usetex": False, "mathtext.fontset": "stix",
    })

device = get_device()
print(f"Using {device} device")
set_mpl_style()

# Metrics
def relative_l2_error(u_num: torch.Tensor, u_ref: torch.Tensor) -> torch.Tensor:
    return torch.norm(u_num - u_ref) / torch.norm(u_ref)

# Util function to plot the solutions
def plot_comparison(t, theta_true, theta_pred, loss):
    t, u, u_hat = (
        x.detach().cpu().numpy().ravel()
        for x in (t, theta_true, theta_pred)
    )

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    ax[0].plot(t, u, label=r'$\theta(t)$ (numerical)')
    ax[0].plot(t, u_hat, label=r'$\theta_{\mathrm{pred}}(t)$')
    ax[0].set(title='Numerical vs Predicted',
              xlabel=r'Time $(s)$', ylabel='Amplitude', ylim=(-1, 1.3))
    ax[0].legend(frameon=False)

    ax[1].plot(t, np.abs(u - u_hat))
    ax[1].set(title='Absolute Difference',
              xlabel=r'Time $(s)$',
              ylabel=r'$|\theta - \theta_{\mathrm{pred}}|$')

    fig.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(6, 3))
    ax.plot(loss)
    ax.set(title='Training Progress',
           xlabel='Iteration', ylabel='Loss',
           xscale='log', yscale='log')
    ax.grid(True)
    fig.tight_layout()
    plt.show()

## 1. Modelo matemático del sistema masa–resorte–amortiguador

El sistema masa–resorte–amortiguador describe el movimiento de una masa $m$ sujeta a un resorte de constante elástica $k$ y un elemento disipador o amortiguador con coeficiente $c$. Su ecuación de movimiento, en ausencia de fuerzas externas, está dada por:

\begin{equation*}
m\,\ddot{x}(t) + c\,\dot{x}(t) + k\,x(t) = 0
\end{equation*}

donde:
- $x(t)$ es el desplazamiento de la masa respecto a su posición de equilibrio,  
- $\dot{x}(t)$ es la velocidad,  
- $\ddot{x}(t)$ es la aceleración.

Dividiendo entre $m$, se obtiene la forma normalizada:

\begin{equation*}
\ddot{x}(t) + 2\zeta\omega_n\,\dot{x}(t) + \omega_n^2\,x(t) = 0
\end{equation*}

con:
\begin{equation*}
\omega_n = \sqrt{\frac{k}{m}} \quad \text{(frecuencia natural no amortiguada)}
\end{equation*}

\begin{equation*}
\zeta = \frac{c}{2\sqrt{km}} \quad \text{(razón de amortiguamiento adimensional)}
\end{equation*}

Definimos los siguientes parámetros con los que vamos a trabajar

In [ ]:
# Dominio temporal
T = 5.0        # tiempo total de simulación
x0 = 1.0       # Posición inicial 
v0 = 0.0       # velocidad incial
wn = 5.0       # Frecuencia natural
zeta = 0.2     # razón de amortiguamiento

### 2. Definición del dominio (temporal) 

En esta etapa se define el dominio sobre el cual la PINN será entrenada. El tiempo $t$ se discretiza en un conjunto de puntos uniformemente espaciados dentro del intervalo $[0, T]$. Estos puntos se usarán como entradas para la red neuronal durante el entrenamiento, mientras que una malla más densa se empleará para evaluar la solución exacta y comparar el desempeño de la PINN.  

> **💡 REMARK!:**  
> Cada punto $t_i \in [0,T],\ i = 1, \dots, N_{sample}$ se conoce como **punto de colocación**, 
> y su nombre proviene de la similitud de las PINNs con los métodos de colocación.

In [ ]:
def crear_dominio_temporal(T, N_train=101, N_eval=1000):
    """Crea el dominio temporal para la PINN."""
    t_train = torch.linspace(0, T, N_train, 
                             device=device, 
                             requires_grad=True).view(-1, 1)  # entrenamiento
    t_eval = torch.linspace(0, T, N_eval, 
                             device=device, 
                             requires_grad=True).view(-1, 1)  # evaluación
    return t_train, t_eval # dominio de evaluación

### 3. Red neuronal como aproximador de la solución

En esta etapa se define una red neuronal que aproxima la solución de la ecuación diferencial mediante un modelo paramétrico. La función desconocida $x(t)$ se reemplaza por una red neuronal dependiente de un conjunto de parámetros $\Theta$:

$$
x_{PINN}(t; \Theta) \approx x_{real}(t).
$$

Los parámetros de la red están dados por los pesos y sesgos de cada capa, agrupados como:

$$
\Theta = \{ W_i, b_i \}_{i=1}^{L},
$$

donde $W_i$ y $b_i$ representan respectivamente los pesos y sesgos de la capa $i$, y $L$ es el número total de capas de la red. Estos parámetros serán ajustados durante el entrenamiento para minimizar el error entre la ecuación física y la salida de la red.

> **💡 REMARK 1:**  
> Para este ejercicio utilizaremos un perceptrón multicapa, con funciones de 
> activación tahn, y una inicialización de pesos con esquema de Glorot.

In [ ]:
# Define a neural network class with user defined layers and neurons
class NeuralNetwork(nn.Module):

    def __init__(self, hlayers = [1, 10, 10, 1]):
        super(NeuralNetwork, self).__init__()

        layers = []
        for i in range(len(hlayers[:-2])):
            layers.append(nn.Linear(hlayers[i], hlayers[i+1]))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hlayers[-2], hlayers[-1]))

        self.layers = nn.Sequential(*layers)
        self.init_params()

    def init_params(self):
        """Xavier Glorot parameter initialization of the Neural Network
        """
        def init_normal(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight) # Xavier
        self.apply(init_normal)

    def forward(self, x):
        return self.layers(x)


### 4. Diferenciación automática en PyTorch 

Antes de continuar será útil aprender a calcular las derivadas de una red neuronal. Para esto utilizaremos la diferenciación automática (*autodiff*), que es una técnica para calcular gradientes de funciones de forma eficiente y precisa mediante el uso de la regla de la cadena. PyTorch registra la forma en que operamos las variables en un gráfico computacional dinámico, que luego le permite calcular derivadas automáticamente al realizar una "propagación hacia atrás" (*backpropagation*). 

In [ ]:
# Util function to calculate tensor gradients with autodiff
def grad(outputs, inputs):
    """Computes the partial derivative of an output with respect
    to an input.
    Args:
        outputs: (N, 1) tensor
        inputs: (N, D) tensor
    """
    return torch.autograd.grad(outputs, inputs,
                        grad_outputs=torch.ones_like(outputs),
                        create_graph=True,
                        )[0]


# Definir tensor de entrada. Si queremos derivar c/r a x necesitamos inicializar con requires_grad=True
x = torch.tensor([1.0, 2.0, 3.0], device=device, requires_grad=True).view(-1,1).float() # (N,1)

# Calcular operación que dependen de x
y = x**2 # (N,1)  

# Calcular derivadas c/r a x 
# grad es un wrapper de torch.autograd
dy_dx = grad(y, x) 

# Calcular derivadas de orden superior
d2y_dx2 = grad(dy_dx, x)  

print("x:", x)
print("y = x^2:", y)
print("dy/dx:", dy_dx)
print("d^2y/dx^2:", d2y_dx2)  

# Esto también funciona para redes neuronales
# test_ANN = NeuralNetwork()

# NNx = test_ANN(x)
# dNNx_dx = grad(NNx, x)
# print("NNx: ", dNNx_dx)
# print("dNNx/dx:", dNNx_dx)

### 5. Función de pérdida informada por la física del problema

Para entrenar la PINN, utilizamos el modelo del sistema masa–resorte–amortiguador en su forma normalizada, donde la frecuencia natural $\omega_n$ y la razón de amortiguamiento $\zeta$ caracterizan la dinámica:

$$
\ddot{x}(t) + 2\zeta\omega_n\,\dot{x}(t) + \omega_n^2 x(t) = 0
$$

Reemplazamos la solución desconocida por la salida de la red neuronal $x_{PINN}(t;\Theta)$ y definimos el residuo físico y las condiciones iniciales:

$$
\begin{align*}
f_{ode}(t;\,x_{PINN}) := &\;
\frac{d^2x_{PINN}(t; \Theta)}{dt^2}
+ 2\zeta\omega_n \frac{dx_{PINN}(t; \Theta)}{dt}
+ \omega_n^2 x_{PINN}(t; \Theta) = 0, \\
g_{ic}(0;\,x_{PINN}) :=&\;
x_{PINN}(0;\Theta) - x_0 = 0, \\
h_{ic}(0;\,x_{PINN}) :=&\;
\frac{dx_{PINN}(0;\Theta)}{dt} - v_0 = 0.
\end{align*}
$$

La función de pérdida total que se optimiza es:

$$
\begin{align*}
\mathcal{L}(\Theta) := &
\frac{\lambda_1}{N}\sum_i \left( f_{ode}(t_i;\,x_{PINN} - 0) \right)^2\\
+ &\lambda_2 \left( g_{ic}(0;\,x_{PINN}) - x_0 \right)^2\\
+ &\lambda_3 \left( h_{ic}(0;\,x_{PINN}) - v_0 \right)^2.
\end{align*}
$$

El entrenamiento busca minimizar esta función:

$$
\min_{\Theta} \mathcal{L}(\Theta) \rightarrow 0
$$

utilizando diferenciación automática (`torch.autograd`) para calcular las derivadas de la red con respecto al tiempo, necesarias para evaluar el residuo físico del modelo.

> **💡 REMARK:**  
> Cuando no incluimos la función de pérdida relacionada con los datos, estamos 
> empleando un esquema independiente de datos (*data-free*); cuando incluimos una función de pérdida relacionada a los datos, estamos empleando un esquema basado en datos (*data-driven*).


> **💡 REMARK:**  
> En este esquema, las condiciones iniciales y de frontera se cumplen débilmente. 

In [ ]:
# Define a loss function (Mean Squared Error) for training the network
MSE_func = nn.MSELoss()

# derivatives of the ANN
def PINNLoss(PINN, t_phys, wn, zeta, x0 = 1, v0 = 0,
             lambda1 = 1, lambda2 = 1, lambda3 = 1):

    t0 = torch.tensor(0., device=device, requires_grad=True).view(-1,1)

    # ANN output, first and second derivatives
    x_pinn_t = PINN(t_phys)
    x_pinn_dt = grad(x_pinn_t, t_phys)
    x_pinn_ddt = grad(x_pinn_dt, t_phys)
    
    f_ode = x_pinn_ddt + 2 * zeta * wn * x_pinn_dt + wn**2 * x_pinn_t
    ODE_loss = lambda1 * MSE_func(f_ode, torch.zeros_like(f_ode)) 
    
    g_ic = PINN(t0)
    IC_loss = lambda2 * MSE_func(g_ic, torch.ones_like(g_ic)*x0)
    
    h_bc = grad(PINN(t0),t0)
    BC_loss = lambda3 * MSE_func(h_bc, torch.zeros_like(h_bc)*v0)
    
    return ODE_loss + IC_loss + BC_loss 

### 6. Definición del optimizador

Una vez definida la función de pérdida, se selecciona un optimizador para ajustar los parámetros $\Theta = \{W_i, b_i\}$ de la red neuronal. El objetivo del optimizador es minimizar la función de pérdida informada por la física, es decir:

$$
\min_{\Theta} \mathcal{L}(\Theta) \quad \text{via} \quad \Theta^{k+1}=\Theta^{k}-\alpha\nabla_\Theta\mathcal{L}(\Theta^{k})
$$

En esta etapa se emplea el optimizador **ADAM**, un método de descenso de gradiente adaptativo que ajusta dinámicamente la tasa de aprendizaje para cada parámetro. ADAM combina las ventajas del momento clásico y del escalado adaptativo de gradientes, lo que lo hace especialmente adecuado para el entrenamiento de PINNs.

In [ ]:
def pinn_optimizer(pinn, lr = 0.01):

    # Define an optimizer (Adam) for training the network
    return optim.Adam(pinn.parameters(), lr=lr,
                        betas= (0.99,0.999), eps = 1e-8)

### Ciclo de entrenamiento

En esta etapa se ejecuta el proceso iterativo mediante el cual la PINN ajusta sus parámetros $\Theta = \{W_i, b_i\}$ para minimizar la función de pérdida definida anteriormente. Durante cada época (*epoch*), el modelo evalúa el residuo de la ecuación diferencial y las condiciones iniciales sobre los puntos de colocación $t_i \in [0, T]$, actualizando los parámetros según el gradiente de la pérdida:

$$
\Theta \leftarrow \Theta - \eta \, \nabla_\Theta \mathcal{L}(\Theta)
$$

donde $\eta$ es la tasa de aprendizaje. El ciclo continúa hasta que la pérdida alcanza un valor suficientemente pequeño o deja de mejorar significativamente. Finalmente, la solución obtenida $x_{PINN}(t; \Theta)$ se compara con la solución exacta para evaluar la calidad del entrenamiento y la capacidad del modelo de reproducir la dinámica del sistema masa–resorte–amortiguador.

A continuación se presenta el código completo:

In [ ]:
#===============================================================================
# ETAPA 1: DEFINICIÓN DE LOS PARÁMETROS (MODELO FÍSICO)
#===============================================================================
# Dominio temporal
T = 5.0        # tiempo total de simulación
x0 = 1.0       # Posición inicial 
v0 = 0.0       # velocidad incial
wn = 5.0  # Frecuencia natural
zeta = 0.2     # razón de amortiguamiento

#===============================================================================
# ETAPA 2: DEFINICIÓN DEL DOMINIO 
#===============================================================================
# Creamos los tensores de tiempo para el entrenamiento y la evaluación
t_train, t_eval = crear_dominio_temporal(T)

#===============================================================================
# ETAPA 3: CREACIÓN DE LA RED NEURONAL SURROGANTE 
#===============================================================================
# Creamos la ANN
torch.manual_seed(123)
hidden_layers = [1, 30, 30, 30, 1]# Parámetros de la 

# Create an instance of the neural network
x_pinn = NeuralNetwork(hidden_layers).to(device)
nparams = sum(p.numel() for p in x_pinn.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {nparams}')

#==========================================================================
# ETAPA 4 Y 5: DEFINICIÓN DE LA FUNCIÓN DE COSTO BASADA EN AUTOGRAD
#==========================================================================
# Define a loss function (Mean Squared Error) for training the network
MSE_func = nn.MSELoss()

# derivatives of the ANN
def PINNLoss(PINN, t_phys, wn, zeta, x0 = 1, v0 = 0, 
             w1 = 1, w2 = 1, w3 = 1):

    t0 = torch.tensor(0., device=device, requires_grad=True).view(-1,1)

    # ANN output, first and second derivatives
    x_pinn_t = PINN(t_phys)
    x_pinn_dt = grad(x_pinn_t, t_phys)
    x_pinn_ddt = grad(x_pinn_dt, t_phys)
    
    f_ode = x_pinn_ddt + 2 * zeta * wn * x_pinn_dt + wn**2 * x_pinn_t
    ODE_loss = w1 * MSE_func(f_ode, torch.zeros_like(f_ode)) 
    
    g_ic = PINN(t0)
    IC_loss = w2 * MSE_func(g_ic, torch.ones_like(g_ic)*x0)
    
    h_bc = grad(PINN(t0),t0)
    BC_loss = w3 * MSE_func(h_bc, torch.zeros_like(h_bc)*v0)
    
    return ODE_loss + IC_loss + BC_loss 

#==========================================================================
# ETAPA 6: DEFINICIÓN DEl OPTIMIZADOR
#==========================================================================
lr = 0.01
optimizer = pinn_optimizer(x_pinn, lr)

#==========================================================================
# CICLO DE ENTRENAMIENTO
#==========================================================================
training_iter = 20000

# Initialize a list to store the loss values
loss_values_pinn = []

# Start the timer
start_time = time.time()

# Training the neural network
for i in range(training_iter):

    optimizer.zero_grad()   # clear gradients for next train

    # input x and predict based on x
    loss = PINNLoss(x_pinn, t_train, wn, zeta)

    # Append the current loss value to the list
    loss_values_pinn.append(loss.item())

    if i % 500 == 0:  # print every 100 iterations
        print(f"Iteration {i}: Loss {loss.item()}")

    loss.backward() # compute gradients (backpropagation)
    optimizer.step() # update the ANN weigths

# Stop the timer and calculate the elapsed time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

## Validación

El comportamiento del sistema depende de $\zeta$:

- **Subamortiguado** ($0 < \zeta < 1$): el sistema oscila con frecuencia amortiguada  
  \begin{equation*}
  \omega_d = \omega_n \sqrt{1 - \zeta^2}
  \end{equation*}

  la solución puede escribirse en términos de las condiciones iniciales $x(0)=x_0$ y $\dot{x}(0)=v_0$:
  \begin{equation*}
  x(t) = e^{-\zeta \omega_n t}
  \left[
  x_0 \cos(\omega_d t) +
  \frac{v_0 + \zeta \omega_n x_0}{\omega_d}
  \sin(\omega_d t)
  \right]
  \end{equation*}

- **Críticamente amortiguado** ($\zeta = 1$): no hay oscilaciones, el sistema retorna a equilibrio lo más rápido posible sin sobrepasarlo:
  \begin{equation*}
  x(t) = (x_0 + (v_0 + \omega_n x_0)t)\, e^{-\omega_n t}
  \end{equation*}

- **Sobreamortiguado** ($\zeta > 1$): el sistema retorna al equilibrio sin oscilar, pero más lentamente:
  \begin{equation*}
  x(t) =
  \frac{v_0 - r_2 x_0}{r_1 - r_2} e^{r_1 t} +
  \frac{r_1 x_0 - v_0}{r_1 - r_2} e^{r_2 t}
  \end{equation*}

  con:
  \begin{equation*}
  r_{1,2} = -\omega_n\left( \zeta \pm \sqrt{\zeta^2 - 1} \right)
  \end{equation*}


Esta expresión servirá como referencia para validar la solución obtenida mediante la red neuronal informada por la física.


In [ ]:
def masa_resorte_general(t, x0, v0, omega_n, zeta):
    """
    Solución exacta x(t) del sistema masa-resorte-amortiguador:
        x'' + 2*zeta*omega_n*x' + omega_n^2*x = 0
    Incluye los tres regímenes (sub, crítico y sobreamortiguado).
    """
    t = np.array(t, dtype=float)

    if 0 < zeta < 1:  # Subamortiguado
        omega_d = omega_n * np.sqrt(1 - zeta**2)
        x = np.exp(-zeta * omega_n * t) * (
            x0 * np.cos(omega_d * t) +
            (v0 + zeta * omega_n * x0) / omega_d * np.sin(omega_d * t)
        )

    elif np.isclose(zeta, 1.0):  # Críticamente amortiguado
        x = np.exp(-omega_n * t) * (
            x0 * (1 + omega_n * t) + v0 * t
        )

    elif zeta > 1:  # Sobreamortiguado
        r1 = -omega_n * (zeta - np.sqrt(zeta**2 - 1))
        r2 = -omega_n * (zeta + np.sqrt(zeta**2 - 1))
        x = (
            ((v0 - r2 * x0) / (r1 - r2)) * np.exp(r1 * t) +
            ((r1 * x0 - v0) / (r1 - r2)) * np.exp(r2 * t)
        )

    else:
        raise ValueError("zeta debe ser mayor que 0.")

    return torch.tensor(x, dtype=torch.float32, device=device).view(-1, 1)

# -------------------------------------------
# solucion exacta
t_eval_np = t_eval.detach().cpu().numpy().ravel()
x = masa_resorte_general(t_eval_np, x0, v0, wn, zeta)

# predicción de la PINN
x_pred_pinn = x_pinn(t_eval)

print(f'Relative error: {relative_l2_error(x_pred_pinn, x)}')

plot_comparison(t_eval, x, x_pred_pinn, loss_values_pinn)


## **Ejercicios**:
1. Ajusta los valores de los parámetros del sistema dinámico y analiza su impacto en la convergencia de la PINN.
2. Ajusta los valores de los parámetros `lambdas` en la función de pérdida para ambas redes y analiza su impacto.
2. Modifica la tasa de aprendizaje (`learning_rate`) del optimizador y el número de iteraciones de entrenamiento, y evalúa el efecto en el desempeño.
3. Cambia el número de capas ocultas (`hidden_layers`), neuronas y funciones de activación de la red neuronal, y observa el impacto en los resultados.